In [1]:
import sys
sys.path.append("../")

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
#import pandas as pd
import numpy as np
import cv2
#from PIL import Image
import random
#import tensorflow as tf
#import re
import datetime
import io
#from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import pickle
#import string
from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense, Conv2D, MaxPooling2D, Reshape, Dropout, BatchNormalization, Activation
from utils import score_prediction
import yaml 
import json
from preprocessing.preproc_functions import read_image, normalize_0_mean_1_variance, normalize_0_1, read_annotation, convert_annotation_one_hot
from predictors.predictor import PredictorFCN
from data_generators.data_generator import DataGenerator
from utils.score_prediction import score_prediction 
from keras.applications.vgg16 import preprocess_input

%matplotlib inline

Using TensorFlow backend.


In [4]:
with open('../configs/config.yml') as f:
    config = yaml.load(f)
config['labels_file'] = '../' + config['labels_file']
config['dataset_folder'] = '../' + config['dataset_folder']

config['network']['graph_path'] = '../' + config['network']['graph_path']
config['predict']['weights_file'] = '../' + config['predict']['weights_file']

y_size = config['image']['image_size']['y_size']
x_size = config['image']['image_size']['x_size']
num_classes = config['network']['num_classes']

In [5]:
with open(config['labels_file']) as f:
    dataset = json.load(f)

In [37]:
dataset_test = dataset['train']

In [38]:
len(dataset_test)

2358

In [39]:
predictor = PredictorFCN(config)

In [99]:
index = random.randint(0, len(dataset_test) - 1)
filename = dataset_test[index]['filename']
annotation_name = dataset_test[index]['annotation']

In [100]:
image_orig = read_image(config['dataset_folder'], filename, y_size, x_size, black_white = False)
#image = normalize_0_mean_1_variance(image_orig)
image = preprocess_input(image_orig, mode='tf') 

annotation = read_annotation(config['dataset_folder'], annotation_name, y_size, x_size)
annotation_one_hot = convert_annotation_one_hot(annotation, y_size, x_size, num_classes = config['network']['num_classes'])
annotation = np.argmax(annotation_one_hot, axis=2)

In [101]:
prediction = predictor.predict(np.expand_dims(image, axis=0))[0]
pred_classes = np.argmax(prediction, axis=2)

In [1]:
fig, axs = plt.subplots(1, 3, figsize=(16, 16))

images = []

axs[0].set_title("Image")
axs[1].set_title("Ground Truth")
axs[2].set_title("Prediction")

images.append(axs[0].imshow(image_orig))
images.append(axs[1].imshow(annotation, cmap=plt.get_cmap('nipy_spectral'), vmin=0, vmax=num_classes))
images.append(axs[2].imshow(pred_classes, cmap=plt.get_cmap('nipy_spectral'), vmin=0, vmax=num_classes))
    
cbar = fig.colorbar(images[2], ax=axs, orientation='horizontal', ticks=[x for x in range(num_classes)], fraction=.1)
cbar.ax.set_xticklabels([value for (key, value) in dataset['labels'].items()], rotation=55)

plt.show()
#plt.savefig("../figures/pred_3.jpg")
#print(dataset['labels'])

NameError: name 'plt' is not defined

In [13]:
#dataset['labels']

### Predict on full set

In [69]:
test_generator = DataGenerator(config, dataset['test'], shuffle=False, use_data_augmentation=False)

#numpy array containing images
images_test, labels_test = test_generator.get_full_dataset()

In [70]:
images_test.shape

(292, 224, 224, 3)

In [71]:
pred_test = predictor.predict(images_test)

In [72]:
pixel_accuracy, mean_accuracy, mean_IoU, freq_weighted_mean_IoU = score_prediction(labels_test, pred_test, num_classes)

In [73]:
print("pixel accuracy:", round(pixel_accuracy, 2))
print("mean accuracy:", round(mean_accuracy, 2))
print("mean IoU:", round(mean_IoU, 2))
print("freq weighted mean IoU:", round(freq_weighted_mean_IoU, 2))

pixel accuracy: 0.81
mean accuracy: 0.35
mean IoU: 0.27
freq weighted mean IoU: 0.69
